<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e278a75c87b545cdf0fc8846642552fa927a405fcd3c14550bf9ec57b0708f69
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-15 10:32:08
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.36 C
-------------------
Today PnL: 12.58 K (0.09%)
Current PnL: -28.15 L (-18.18%)
CY Booked + Current PnL: -13.86 L (-8.95%)
-------------------
Total profit:  1.39 L
Total loss:  -29.54 L
-------------------
Total Booked + Current PnL: 13.12 L (10.27%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.52%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.29 C
Est FTT PnL: 92.78 L (68.3%)
Deployed:  1.28 C
Current:  1.36 C
CAGR/XIRR %: 3.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.17,-16.59,20.00,0.09,16811.0,-16721.0,84056.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.62,10.43,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,1.03,-11.45,12.95,0.02,20018.0,-19981.0,154578.0,135.49,55.0,M-SC,15.39,234.0,-1.00,1.14,34.46,OX40N,NTT,PAINTS
50,MASFIN,398.61,1.19,0.09,21.94,22.05,21517.0,90.0,98070.0,-13.73,61.0,H-SC,7.21,164.0,0.00,0.73,42.55,XR,ATH,FINANCE
82,VOLTAS,1530.00,-0.33,7.39,11.46,19.69,23597.0,14163.0,205905.0,-3.15,53.0,X-MC,2.54,78.0,0.60,1.52,14.69,XY25,NTT,AC
43,ITC,452.00,0.10,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,43.0,X-LC,1.93,5.0,-0.17,1.45,3.30,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,10.34,-39.21,122.53,35.27,84575.0,-44525.0,69024.0,-53.72,51.0,L-SC,10.87,270.0,-0.53,0.51,68.94,XR,NTT,HOTELS
57,RAJOOENG,143.1,2.96,-36.24,118.97,39.61,77747.0,-37150.0,65350.0,-56.80,30.0,H-SC,28.16,135.0,-0.48,0.48,2.96,AR,ATH,MISC
44,JCHAC,2282.0,2.28,-36.03,56.30,-0.01,46031.0,-46045.0,81760.0,16639.13,30.0,M-SC,6.68,233.0,-1.00,0.60,4.64,OX40N,NTT,AC
79,VAIBHAVGBL,521.0,1.95,-25.69,116.34,60.77,158015.0,-46953.0,135822.0,56.47,50.0,H-SC,4.30,159.0,-0.30,1.00,22.32,XR,NTT,JEWELLERY
0,5PAISA,593.0,1.93,-40.24,88.91,12.90,116100.0,-87914.0,130582.0,92.31,47.0,H-SC,12.03,161.0,-0.76,0.97,8.58,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,806.63,-1.58,-16.15,66.42,39.54,56339.0,-16337.0,84822.0,-34.36,48.0,M-SC,4.62,220.0,-0.29,0.63,19.61,AR,ATH,AUTO
6,ASIANTILES,137.00,-1.48,-16.99,118.85,81.67,92553.0,-15936.0,77874.0,7000.00,44.0,L-SC,13.01,271.0,-0.17,0.58,50.81,XR,NTT,CERAMICS
14,CAMPUS,393.00,-1.44,-11.98,51.42,33.28,75939.0,-20091.0,147684.0,-29.13,36.0,M-SC,2.63,222.0,-0.26,1.09,14.47,XY24,NTT,FOOTWEAR
58,RECLTD,446.00,-1.28,-7.96,31.18,20.74,58307.0,-16170.0,187000.0,21.78,32.0,M-MC,7.43,189.0,-0.28,1.38,0.00,XY25,NTT,FINANCE
16,CERA,9475.00,-1.27,-32.12,82.51,23.89,98520.0,-56499.0,119404.0,-34.61,28.0,H-SC,10.30,157.0,-0.57,0.88,3.12,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,1.19,0.09,21.94,22.05,21517.0,90.0,98070.0,-13.73,61.0,H-SC,7.21,164.0,0.0,0.73,42.55,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.79,-5.51,120.05,107.91,167472.0,-8142.0,139502.0,-22.98,45.0,M-SC,12.66,216.0,-0.05,1.03,4.02,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.03,-11.45,12.95,0.02,20018.0,-19981.0,154578.0,135.49,55.0,M-SC,15.39,234.0,-1.00,1.14,34.46,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.17,-16.59,20.00,0.09,16811.0,-16721.0,84056.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.62,10.43,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.66,-24.91,51.12,13.47,103509.0,-67185.0,202482.0,-69.09,34.0,H-SC,4.77,158.0,-0.65,1.50,2.96,XY24,NTT,PAINTS
66,SIS,528.00,-0.11,-25.47,62.71,21.26,51891.0,-28284.0,82748.0,1950.26,43.0,H-SC,4.94,166.0,-0.55,0.61,11.80,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.52,6.89,61.17,72.27,110789.0,11669.0,181117.0,-8.35,68.0,M-LC,2.02,99.0,0.11,1.34,13.49,XR,NTT,IT
50,MASFIN,398.61,1.19,0.09,21.94,22.05,21517.0,90.0,98070.0,-13.73,61.0,H-SC,7.21,164.0,0.00,0.73,42.55,XR,ATH,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.52,6.89,61.17,72.27,110789.0,11669.0,181117.0,-8.35,68.0,M-LC,2.02,99.0,0.11,1.34,13.49,XR,NTT,IT
50,MASFIN,398.61,1.19,0.09,21.94,22.05,21517.0,90.0,98070.0,-13.73,61.0,H-SC,7.21,164.0,0.00,0.73,42.55,XR,ATH,FINANCE
38,INDIAMART,4810.62,1.16,-2.97,113.05,106.72,135291.0,-3662.0,119674.0,-51.96,37.0,H-SC,2.68,139.0,-0.03,0.89,19.71,AR,ATH,MISC
37,IEX,219.00,0.32,-2.50,52.44,48.63,102230.0,-5007.0,194947.0,-35.06,53.0,H-SC,14.02,136.0,-0.05,1.44,8.76,XR,NTT,MISC
25,FINCABLES,1641.55,0.79,-5.51,120.05,107.91,167472.0,-8142.0,139502.0,-22.98,45.0,M-SC,12.66,216.0,-0.05,1.03,4.02,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.20,-22.72,92.10,48.45,142706.0,-45559.0,154947.0,-28.25,22.0,X-MC,12.50,64.0,-0.32,1.15,0.89,X40N,ATH,FINANCE
75,TMPV,600.00,-0.71,-18.25,73.91,42.18,167018.0,-50435.0,225975.0,-26.74,29.0,X-LC,6.49,3.0,-0.30,1.67,0.47,XY24,NTT,AUTO
53,PAGEIND,51605.07,-0.43,-10.17,40.35,26.07,59347.0,-16660.0,147080.0,-33.88,31.0,X-MC,12.66,55.0,-0.28,1.09,0.20,X40,ATH,APPARELS
33,HINDUNILVR,2922.00,0.99,-11.19,27.99,13.67,63901.0,-28763.0,228300.0,-17.78,31.0,X-LC,2.17,14.0,-0.45,1.69,7.78,XY25,NTT,FMCG
51,MEDANTA,1486.00,-0.93,-4.67,30.16,24.08,36500.0,-5930.0,121020.0,-12.39,32.0,X-SC,5.24,89.0,-0.16,0.90,14.03,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-0.42,-80.29,533.24,24.79,264658.0,-202224.0,49632.0,-81.32,43.0,X-SC,0.67,86.0,-0.76,0.37,21.33,X40N,NTT,MISC
43,ITC,452.0,0.10,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,43.0,X-LC,1.93,5.0,-0.17,1.45,3.30,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.73,6.56,22.90,30.97,43130.0,11602.0,188340.0,-18.51,61.0,X-MC,2.10,75.0,0.27,1.39,19.95,X40,ATH,INSURANCE
33,HINDUNILVR,2922.0,0.99,-11.19,27.99,13.67,63901.0,-28763.0,228300.0,-17.78,31.0,X-LC,2.17,14.0,-0.45,1.69,7.78,XY25,NTT,FMCG
3,ACC,3906.0,0.72,-25.01,118.90,64.15,212165.0,-59511.0,178440.0,-56.31,38.0,X-SC,2.40,82.0,-0.28,1.32,0.72,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.43,-10.17,40.35,26.07,59347.0,-16660.0,147080.0,-33.88,31.0,X-MC,12.66,55.0,-0.28,1.09,0.20,X40,ATH,APPARELS
75,TMPV,600.00,-0.71,-18.25,73.91,42.18,167018.0,-50435.0,225975.0,-26.74,29.0,X-LC,6.49,3.0,-0.30,1.67,0.47,XY24,NTT,AUTO
3,ACC,3906.00,0.72,-25.01,118.90,64.15,212165.0,-59511.0,178440.0,-56.31,38.0,X-SC,2.40,82.0,-0.28,1.32,0.72,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-1.20,-22.72,92.10,48.45,142706.0,-45559.0,154947.0,-28.25,22.0,X-MC,12.50,64.0,-0.32,1.15,0.89,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.18,-38.17,118.68,35.20,94417.0,-49114.0,79556.0,4.26,34.0,X-SC,15.99,92.0,-0.52,0.59,1.05,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.72,-29.93,103.85,42.84,47306.0,-19454.0,45552.0,-54.27,41.0,X-SC,35.94,83.0,-0.41,0.34,3.06,XY24,NTT,MISC
15,CAMS,4762.00,-0.42,-80.29,533.24,24.79,264658.0,-202224.0,49632.0,-81.32,43.0,X-SC,0.67,86.0,-0.76,0.37,21.33,X40N,NTT,MISC
59,RELAXO,1176.00,0.09,-47.50,190.08,52.31,144869.0,-68945.0,76215.0,-44.15,43.0,X-SC,6.70,91.0,-0.48,0.56,2.02,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.18,-38.17,118.68,35.20,94417.0,-49114.0,79556.0,4.26,34.0,X-SC,15.99,92.0,-0.52,0.59,1.05,X40,NTT,FOOTWEAR
22,DIXON,18931.72,1.52,-12.77,39.38,21.58,48141.0,-17892.0,122247.0,-56.03,40.0,X-MC,8.36,56.0,-0.37,0.90,9.97,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.16,-12.35,36.09,19.28,109434.0,-42731.0,303225.0,-23.62,64.0,X-LC,4.66,1.0,-0.39,2.24,12.10,X40,ATH,IT
41,INFY,2275.00,0.49,9.34,41.65,54.87,142484.0,29211.0,342099.0,-14.93,64.0,X-LC,3.59,2.0,0.21,2.53,18.39,X40,BTT,IT
75,TMPV,600.00,-0.71,-18.25,73.91,42.18,167018.0,-50435.0,225975.0,-26.74,29.0,X-LC,6.49,3.0,-0.30,1.67,0.47,XY24,NTT,AUTO
81,VBL,671.64,-0.88,-3.90,41.17,35.67,124962.0,-12314.0,303528.0,-15.30,55.0,X-LC,4.49,4.0,-0.10,2.25,9.42,X40N,ATH,FMCG
43,ITC,452.00,0.10,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,43.0,X-LC,1.93,5.0,-0.17,1.45,3.30,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.56,-37.88,113.93,32.89,54641.0,-29246.0,47960.0,-692.31,53.0,L-MC,5.92,259.0,-0.54,0.35,32.08,XR,NTT,BANKS
6,ASIANTILES,137.00,-1.48,-16.99,118.85,81.67,92553.0,-15936.0,77874.0,7000.00,44.0,L-SC,13.01,271.0,-0.17,0.58,50.81,XR,NTT,CERAMICS
50,MASFIN,398.61,1.19,0.09,21.94,22.05,21517.0,90.0,98070.0,-13.73,61.0,H-SC,7.21,164.0,0.00,0.73,42.55,XR,ATH,FINANCE
13,BSOFT,831.70,0.64,-21.60,92.32,50.79,101806.0,-30378.0,110275.0,0.50,68.0,H-SC,6.08,151.0,-0.30,0.82,28.40,XR,ATH,IT
39,INDIGOPNTS,1408.00,1.03,-11.45,12.95,0.02,20018.0,-19981.0,154578.0,135.49,55.0,M-SC,15.39,234.0,-1.00,1.14,34.46,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.64,-21.60,92.32,50.79,101806.0,-30378.0,110275.0,0.50,68.0,H-SC,6.08,151.0,-0.30,0.82,28.40,XR,ATH,IT
84,WIPRO,420.00,0.52,6.89,61.17,72.27,110789.0,11669.0,181117.0,-8.35,68.0,M-LC,2.02,99.0,0.11,1.34,13.49,XR,NTT,IT
41,INFY,2275.00,0.49,9.34,41.65,54.87,142484.0,29211.0,342099.0,-14.93,64.0,X-LC,3.59,2.0,0.21,2.53,18.39,X40,BTT,IT
50,MASFIN,398.61,1.19,0.09,21.94,22.05,21517.0,90.0,98070.0,-13.73,61.0,H-SC,7.21,164.0,0.00,0.73,42.55,XR,ATH,FINANCE
32,HCLTECH,1922.01,0.32,8.93,14.50,24.73,38213.0,21616.0,263540.0,13.61,66.0,X-LC,6.71,7.0,0.57,1.95,25.53,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.81
1,25,45.64
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.20
MC    30.19
LC    24.62
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.26
X40      22.35
X40N     10.85
XR       10.22
XY25      9.38
AR        9.33
OX40N     7.80
SR        1.02
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.76
X-MC    23.49
X-LC    21.06
M-SC    12.09
X-SC     6.88
H-MC     4.97
L-SC     1.47
M-MC     1.38
M-LC     1.34
H-LC     1.17
L-LC     1.05
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.28,-7.81,42.67
IT,12.85,-15.93,78.61
FINANCE,9.59,-19.67,70.24
ELECTRICAL,6.23,-11.47,52.29
MISC,6.10,-57.77,122.40
PAINTS,6.08,-12.43,29.13
INSURANCE,4.62,-1.26,36.95
PHARMA,4.06,-1.58,34.09
AUTO,3.40,-24.98,78.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3217943.0,21
AR,1310853.0,10
XR,1282470.0,13
X40,1016820.0,14
X40N,982111.0,9
OX40N,736258.0,10
XY25,373205.0,6
SR,281999.0,2
MH,76225.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3633490.0,25
M-SC,1423762.0,15
X-MC,1307210.0,16
X-SC,990817.0,8
X-LC,916442.0,11
H-MC,406156.0,3
L-SC,263148.0,3
M-LC,110789.0,1
H-LC,75241.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1268217.0      6
           AR         897309.0      5
M-SC       XY24       816662.0      6
H-SC       XR         766143.0      7
X-SC       X40N       506602.0      3
X-MC       X40        454285.0      7
           XY24       399675.0      3
X-LC       X40        374291.0      5
H-SC       OX40N      343597.0      4
M-SC       OX40N      306641.0      5
X-SC       XY24       295971.0      3
H-SC       SR         281999.0      2
X-MC       X40N       272716.0      4
X-LC       XY24       242875.0      2
H-MC       AR         211613.0      2
X-LC       X40N       202793.0      2
H-MC       XY24       194543.0      1
X-SC       X40        188244.0      2
X-MC       XY25       180534.0      2
L-SC       XR         177128.0      2
M-SC       XR         173769.0      2
           AR         126690.0      2
M-LC       XR         110789.0      1
X-LC       XY25        96483.0      2
L-SC       OX40N       86020.0      1
H-SC       MH          76225.0      1
H-LC       AR          75241.0      1
M-MC       XY25        58307.0      1
L-MC       XR          54641.0      1
L-LC       XY25        37881.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
